In [ ]:
#Importing
import numpy as np
import random
import time
import warnings

# Suppress runtime warnings that can occur in benchmark functions
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
#######BENCHMARKING##############
def F1(x): return np.sum(x**2)
def F2(x): return np.sum(np.abs(x)) + np.prod(np.abs(x))
def F3(x):
    s = 0
    for i in range(len(x)): s += (np.sum(x[0:i+1]))**2
    return s
def F4(x): return np.max(np.abs(x))
def F5(x):
    dim = len(x)
    return np.sum(100 * (x[1:dim] - (x[0:dim-1]**2))**2 + (x[0:dim-1] - 1)**2)
def F6(x): return np.sum((np.floor(x + 0.5))**2)
def F7(x): return np.sum(np.arange(1, len(x)+1) * (x**4)) + np.random.uniform(0, 1)
def F8(x): return np.sum(-x * np.sin(np.sqrt(np.abs(x))))
def F9(x): return np.sum(x**2 - 10 * np.cos(2 * np.pi * x) + 10)
def F10(x):
    n = len(x)
    return -20*np.exp(-0.2*np.sqrt(np.sum(x**2)/n)) - np.exp(np.sum(np.cos(2*np.pi*x))/n) + 20 + np.e
def F11(x):
    return (1/4000) * np.sum(x**2) - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x)+1)))) + 1
def u_penalty_function(val, a, k=100, m=4):
    """
    The penalty function u() as defined in the paper for F12 and F13.
    This is a direct mathematical translation without custom precision handling.
    """
    if val > a:
        return k * ((val - a) ** m)
    elif val < -a:
        return k * ((-val - a) ** m)
    else:
        return 0.0

def F12(x):
    """F12 implementation using the original u() penalty function."""
    n = len(x)
    y = 1 + (x + 1) / 4
    term1 = 10 * np.sin(np.pi * y[0])**2
    term2 = np.sum((y[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * y[1:])**2))
    term3 = (y[-1] - 1)**2
    penalty = np.sum([u_penalty_function(val, 10) for val in x])
    return (np.pi / n) * (term1 + term2 + term3) + penalty

def F13(x):
    """F13 implementation using the original u() penalty function."""
    n = len(x)
    term1 = 0.1 * np.sin(3 * np.pi * x[0])**2
    term2 = np.sum((x[:-1] - 1)**2 * (1 + np.sin(3 * np.pi * x[1:])**2))
    term3 = (x[-1] - 1)**2 * (1 + np.sin(2 * np.pi * x[-1])**2)
    penalty = np.sum([u_penalty_function(val, 5) for val in x])
    return term1 + 0.1 * (term2 + term3) + penalty
def F14(x):
    a = np.array([[-32, -16, 0, 16, 32]*5, [-32]*5 + [-16]*5 + [0]*5 + [16]*5 + [32]*5])
    s = sum(1 / (j + 1 + (x[0] - a[0, j])**6 + (x[1] - a[1, j])**6) for j in range(25))
    return (1/500 + s)**-1
def F15(x):
    a = np.array([0.00030,0.00138,0.00138,0.00138,0.00138,0.00030,0.00138,0.00138,0.00138,0.00138,0.00030])
    b = np.array([-2.477,-2.477,-2.477,-2.477,-2.477,2.477,2.477,2.477,2.477,2.477,2.477])
    return np.sum((a - x[0]*(b**2 + b*x[1]) / (b**2 + b*x[2] + x[3]))**2)
def F16(x): return 4*x[0]**2 - 2.1*x[0]**4 + (1/3)*x[0]**6 + x[0]*x[1] - 4*x[1]**2 + 4*x[1]**4
def F17(x): return (x[1]-(5.1/(4*np.pi**2))*x[0]**2+(5/np.pi)*x[0]-6)**2+10*(1-(1/(8*np.pi)))*np.cos(x[0])+10
def F18(x):
    return (1+(x[0]+x[1]+1)**2*(19-14*x[0]+3*x[0]**2-14*x[1]+6*x[0]*x[1]+3*x[1]**2)) * \
           (30+(2*x[0]-3*x[1])**2*(18-32*x[0]+12*x[0]**2+48*x[1]-36*x[0]*x[1]+27*x[1]**2))
def F19(x):
    a = np.array([[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]])
    p = np.array([[0.3689, 0.1170, 0.2673], [0.4699, 0.4387, 0.7470], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])
    c = np.array([1, 1.2, 3, 3.2])
    return -np.sum(c * np.exp(-np.sum(a * (x - p)**2, axis=1)))
def F20(x):
    a = np.array([[10,3,17,3.5,1.7,8], [0.05,10,17,0.1,8,14], [3,3.5,1.7,10,17,8], [17,8,0.05,10,0.1,14]])
    p = np.array([[0.1312,0.1696,0.5569,0.0124,0.8283,0.5886],[0.2329,0.4135,0.8307,0.3736,0.1004,0.9991],
                  [0.2348,0.1415,0.3522,0.2883,0.3047,0.6650],[0.4047,0.8828,0.8732,0.5743,0.1091,0.0381]])
    c = np.array([1, 1.2, 3, 3.2])
    return -np.sum(c * np.exp(-np.sum(a * (x - p)**2, axis=1)))
def _shekel(x, a, c, m): return sum(-1 / (np.sum((x - a[i])**2) + c[i]) for i in range(m))
def F21(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7]]); c=np.array([0.1,0.2,0.2,0.4,0.4])
    return _shekel(x,a,c,5)
def F22(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7],[2,9,2,9],[5,5,3,3]]); c=np.array([0.1,0.2,0.2,0.4,0.4,0.6,0.3])
    return _shekel(x,a,c,7)
def F23(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7],[2,9,2,9],[5,5,3,3],[8,1,8,1],[6,2,6,2],[7,3.6,7,3.6]]); c=np.array([0.1,0.2,0.2,0.4,0.4,0.6,0.3,0.7,0.5,0.5])
    return _shekel(x,a,c,10)

In [ ]:
##############WartOA Implementation####################

def WartOA(obj_func, dim, N, max_iter, lb, ub):

    # Ensure lb and ub are arrays for broadcasting
    if not isinstance(lb, np.ndarray): lb = np.full(dim, lb)
    if not isinstance(ub, np.ndarray): ub = np.full(dim, ub)

    # Initialization (Lines 1-7 of Algorithm 1)
    pop = lb + (ub - lb) * np.random.rand(N, dim)
    fitness = np.array([obj_func(ind) for ind in pop])
    prev_locations = np.copy(pop)

    sw_idx = np.argmin(fitness)
    sw_pos = pop[sw_idx]
    sw_fitness = fitness[sw_idx]

    ages = np.random.randint(1, 16, N)
    time_cycle = ['morning', 'mid-day', 'late-afternoon', 'night']
    weather_conditions = ['hot', 'rainy', 'normal']
    time_idx = 0

    num_burrows = random.randint(15, 25)
    burrows = lb + (ub - lb) * np.random.rand(num_burrows, dim)

    # Main optimization loop (Line 8)
    for t in range(max_iter):
        # Update strongest warthog (Line 9)
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < sw_fitness:
            sw_pos = pop[current_best_idx]
            sw_fitness = fitness[current_best_idx]

        # Regenerate burrows on a fixed schedule (Lines 10-12)
        if (t + 1) % 10 == 0:
            num_burrows = random.randint(15, 25)
            burrows = lb + (ub - lb) * np.random.rand(num_burrows, dim)

        current_time = time_cycle[time_idx % len(time_cycle)]
        weather = random.choice(weather_conditions)
        time_idx += 1

        for i in range(N):
            # Calculate group factor (Eq. 5-6, Line 17)
            group_i = 1.0
            if ages[i] > 2 and np.random.rand() < 0.5:
                group_i = random.uniform(1, 2)
            else:
                group_i = random.uniform(2, 3)

            new_pos = np.copy(pop[i])

            # --- Feeding Strategy (Lines 18-28) ---
            # This block is now corrected to match the paper's pseudocode structure exactly.(*Correction By Subhadipta Daa)
            if current_time in ['morning', 'late-afternoon'] and weather == 'normal':
                feeding_activity = np.random.rand()

                if current_time == 'late-afternoon':
                    # Logic for late-afternoon feeding (Lines 20-22)
                    a = random.randint(1, 3)
                    new_pos = pop[i] + feeding_activity * a * (sw_pos - group_i * pop[i])

                else:
                    # Logic for morning feeding (Lines 24-27)
                    new_pos = pop[i] + feeding_activity * (sw_pos - group_i * pop[i])

                    # The check against the previous location is nested here, as per the paper.
                    if t >= 4:
                        if obj_func(new_pos) > obj_func(prev_locations[i]):
                            new_pos = prev_locations[i] # Revert position (Eq. 9)

            # --- Resting Strategy (Lines 29-32) ---
            elif current_time == 'mid-day' or weather in ['hot', 'rainy']:
                resting_activity = np.random.rand()
                shade = lb + (ub - lb) * np.random.rand(dim)
                new_pos = pop[i] + resting_activity * (shade - pop[i]) # (Eq. 10)

            # --- Seeking Shelter Strategy (Lines 33-39) ---
            elif current_time == 'night':
                prev_locations[i] = np.copy(pop[i])
                distances = np.linalg.norm(pop[i] - burrows, axis=1)
                closest_burrow = burrows[np.argmin(distances)]
                burrow_activity = np.random.rand()
                new_pos = burrow_activity * closest_burrow # (Eq. 11)

            # Update position and fitness (Lines 40-44)
            new_pos = np.clip(new_pos, lb, ub)
            new_fitness = obj_func(new_pos)
            if new_fitness < fitness[i]:
                pop[i], fitness[i] = new_pos, new_fitness

        # Refresh ages periodically (Lines 48-50)
        if (t + 1) % 20 == 0:
            ages = np.random.randint(1, 16, N)

    return sw_pos, sw_fitness

In [ ]:
#Driver COde

if __name__ == '__main__':
    benchmarks = [
        {'name': 'F1', 'func': F1, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F2', 'func': F2, 'dim': 30, 'lb': -10, 'ub': 10},
        {'name': 'F3', 'func': F3, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F4', 'func': F4, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F5', 'func': F5, 'dim': 30, 'lb': -30, 'ub': 30},
        {'name': 'F6', 'func': F6, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F7', 'func': F7, 'dim': 30, 'lb': -1.28, 'ub': 1.28},
        {'name': 'F8', 'func': F8, 'dim': 30, 'lb': -500, 'ub': 500},
        {'name': 'F9', 'func': F9, 'dim': 30, 'lb': -5.12, 'ub': 5.12},
        {'name': 'F10', 'func': F10, 'dim': 30, 'lb': -32, 'ub': 32},
        {'name': 'F11', 'func': F11, 'dim': 30, 'lb': -600, 'ub': 600},
        {'name': 'F12', 'func': F12, 'dim': 30, 'lb': -50, 'ub': 50},
        {'name': 'F13', 'func': F13, 'dim': 30, 'lb': -50, 'ub': 50},
        {'name': 'F14', 'func': F14, 'dim': 2, 'lb': -65.536, 'ub': 65.536},
        {'name': 'F15', 'func': F15, 'dim': 4, 'lb': -5, 'ub': 5},
        {'name': 'F16', 'func': F16, 'dim': 2, 'lb': -5, 'ub': 5},
        {'name': 'F17', 'func': F17, 'dim': 2, 'lb': [-5, 0], 'ub': [10, 15]},
        {'name': 'F18', 'func': F18, 'dim': 2, 'lb': -2, 'ub': 2},
        {'name': 'F19', 'func': F19, 'dim': 3, 'lb': 0, 'ub': 1},
        {'name': 'F20', 'func': F20, 'dim': 6, 'lb': 0, 'ub': 1},
        {'name': 'F21', 'func': F21, 'dim': 4, 'lb': 0, 'ub': 10},
        {'name': 'F22', 'func': F22, 'dim': 4, 'lb': 0, 'ub': 10},
        {'name': 'F23', 'func': F23, 'dim': 4, 'lb': 0, 'ub': 10},
    ]

    population_size = 30
    max_iterations = 500
    num_runs = 30

    print("="*60)
    print("Running Paper Implementation of WartOA")
    print(f"Parameters: Pop Size={population_size}, Max Iterations={max_iterations}, Runs={num_runs}")
    print("="*60)

    for bench in benchmarks:
        name, func, dim, lb, ub = bench.values()
        print(f"\n--- Testing Function: {name} ---")

        results = []
        start_time = time.time()

        for run in range(num_runs):
            _, best_fitness = WartOA(
                obj_func=func,
                dim=dim,
                N=population_size,
                max_iter=max_iterations,
                lb=lb,
                ub=ub
            )
            results.append(best_fitness)
            print(f"  Run {run+1}/{num_runs}, Best Fitness: {best_fitness:.6e}", end='\r')

        end_time = time.time()
        avrg_fitness, std_fitness = np.mean(results), np.std(results)

        print("\n" + "-"*25)
        print(f"Results for {name}:")
        print(f"  AVRG : {avrg_fitness:.6e}")
        print(f"  STD :  {std_fitness:.6e}")
        print(f"  Time Taken:   {end_time - start_time:.2f} seconds")
        print("-"*25)

Running Paper Implementation of WartOA
Parameters: Pop Size=30, Max Iterations=500, Runs=30

--- Testing Function: F1 ---
  Run 30/30, Best Fitness: 1.349429e-71
-------------------------
Results for F1:
  AVRG : 2.151090e-53
  STD :  9.108930e-53
  Time Taken:   11.91 seconds
-------------------------

--- Testing Function: F2 ---
  Run 30/30, Best Fitness: 1.921651e-36
-------------------------
Results for F2:
  AVRG : 2.449457e-27
  STD :  1.318118e-26
  Time Taken:   14.08 seconds
-------------------------

--- Testing Function: F3 ---
  Run 30/30, Best Fitness: 4.066542e-68
-------------------------
Results for F3:
  AVRG : 1.450890e-53
  STD :  7.806819e-53
  Time Taken:   78.09 seconds
-------------------------

--- Testing Function: F4 ---
  Run 30/30, Best Fitness: 2.902934e-36
-------------------------
Results for F4:
  AVRG : 8.508943e-27
  STD :  4.561739e-26
  Time Taken:   11.62 seconds
-------------------------

--- Testing Function: F5 ---
  Run 30/30, Best Fitness: 2.8